In [1]:
# Start pylab inline mode, so figures will appear in the notebook
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# include libs #

In [2]:
from zpickle import *
from bs4 import BeautifulSoup
import re
import os
import os.path
import json
import math
import time
import collections
import operator
import random
import chilin
from joblib import Parallel, delayed
from gensim import corpora, models, similarities
from pprint import pprint
from itertools import cycle
import pylab as pl
import numpy as np

In [3]:
# fn = "./zobj/ALL_DATA_183340.zobj"
# oData = load(fn)

看餐廳的地理參數

Chinese NLP

In [4]:
class zTags():
    ''' 所有的 POS TAG 元素 '''
    @staticmethod
    def getEol():
        EOLS = [
            "FW",
            "QUESTIONCATEGORY",
            "COLONCATEGORY",
            "COMMACATEGORY",
            "DASHCATEGORY",
            "ETCCATEGORY",
            "PARENTHESISCATEGORY",
            "PAUSECATEGORY",
            "PERIODCATEGORY",
            "QUESTIONCATEGORY",
            "SEMICOLONCATEGORY",
            "EXCLANATIONCATEGORY",
            "EXCLAMATIONCATEGORY",
            "BR",
            "SPCHANGECATEGORY"]
        return set(EOLS)
    
    @staticmethod
    def isEol(tag):
        EOLS = zTags.getEol()
        return True if tag in EOLS else False
        
    ''' 過濾出不要的標記 '''
    @staticmethod
    def isWant(tag):
        NOT_WANT = ["^P", "^C", "^D", "^N[c|d|e|f|g|h]", "V_2", "^T", "^SHI"]
        NOT_WANT.extend(zTags.getEol())
        flg = True
        NOT_WANT = set(NOT_WANT)
        for ele in NOT_WANT:
            if re.match(ele, tag):
                flg = False
        return flg
                
    @staticmethod
    def isNot(word):
        ''' 否定句 '''
        # "但是", "但", 
        NOTS = [u"不", u"沒有", u"不要", u"不能", u"沒", u"沒有", u"無", u"不會", u"難", u"算不上", u"未"]
        NOTS = set(NOTS)
        return True if word in NOTS else False
        
    @staticmethod
    def notMorph(word):
        '''帶否定語素'''
        NOTs = u"^[不|難|沒|未]"
        return re.sub(NOTs, "", word) if re.match(NOTs, word) else False

In [5]:
class zWord():
    ''' 基本單詞元素 '''
    def __init__(self, word, go_prev=None, go_next=None):
        self.org = word
        (word, tag) = zWord.toElements(word)
        self.word = word
        
        self.tag = tag
        
        ''' 設定 wanted '''
        self.wanted = zTags.isWant(tag)
        
        ''' 設定 ORI '''
        if zTags.isNot(self.word):
            self.ori = 0
            self.wanted = False
        else:
            self.ori = 1

        # 詞中有否定
        self._selfNot()
            
        # 取得最佳表達式
        self.word_best = self.get_best()

        
    def get_best(self):
        """
            取得最好的表達式， 例如： "不" -> ""； "不用" -> "用-"
        """
        if not self.wanted:
            # 詞類不要
            return ""
        if self.ori == 0:
            # 否定字組
            return ""
        
        return self.word.replace("-", "") if self.ori > 0 else self.word.replace("-", "")+u"-" 
    
    def _selfNot(self):
        '''詞中帶有否定'''
        if not zTags.isNot(self.word) and zTags.notMorph(self.word): # 要改這裡
            if not len(self.word) == 1: # 要改這裡
                '''多語素'''
                self.ori = 0
                # 帶"不"頭
                self.word = zTags.notMorph(self.word)
                self.wanted = True
            else:
                '''單語素'''
                self.ori = 0
                self.word = ""
                self.wanted = False
        
    ''' Toggle Negative '''
    def toggle(self):
        if self.ori == 1:
            self.ori =-1 
        elif self.ori ==-1:
            self.ori = 1 
        
        
    ''' 分開 tag 與 word '''
    @staticmethod
    def toElements(word):
        eles = word.replace(u")", u"").split(u"(")
        if len(eles)==2:
            return (eles[0], eles[1])
        
    def __str__(self):
        return "[%s] %s(%s:%s:%s)"%(self.__class__.__name__, 
                              self.word_best.encode("utf8"),
                              self.tag.encode("utf8"),
                              self.ori,
                              self.wanted
                              )

In [6]:
class zSentence:
    ''' NOTE: 引入的 stn 一定要先用 toWords 切開
        單句元素，會包 zWord 
    '''
    def __init__(self, stn):
        if type(stn)==type([]):
            self.stn_org = stn
        if type(stn)==type(u""):
            self.stn_org = zSentence.toWords(stn)
        self.proc()

    """ 處理評論句子內容 """
    def proc(self):
        words = []
        
        for word in self.stn_org:
            words.append( zWord(word) )
            
        self.words = words
        
        self.notOperation()
        
#         self._define_bigrams()

#     def get_bigrams(self, with_best=False, is_radicals=False):
#         if with_best :
#             if is_radicals:
#                 return self.bigrams_best_radical
#             else:
#                 return self.bigrams_best
#         else:
#             if is_radicals:
#                 return self.bigrams_radical
#             else:
#                 return self.bigrams 
        
#     def _define_bigrams(self, n=5, with_best=False):
#         skipWindow = zSentence.skipWindow
#         self.bigrams_best = set([(tuples[0].get_best(), tuples[1].get_best()) for tuples in skipWindow(self.words, n) if tuples[0].wanted and tuples[1].wanted and not tuples[0].word == tuples[1].word])
#         self.bigrams = set([(tuples[0].word, tuples[1].word) for tuples in skipWindow(self.words, n) if tuples[0].wanted and tuples[1].wanted and not tuples[0].word == tuples[1].word])
        
#         self.bigrams_best_radical = set([ (term2Radi(x), term2Radi(y)) for (x,y) in self.bigrams_best ])
#         self.bigrams_radical = set([ (term2Radi(x), term2Radi(y)) for (x,y) in self.bigrams ])
        
    def get_words(self):
        return [ wrd for wrd in self.words] 
    
    def get_wanted(self):
        ''' 取得需要的字元 '''
        return [ wrd for wrd in self.words if wrd.wanted]
        
    def notOperation(self):
        """ 否定句處理 """
        for idx in range(len(self.words)):
            if self.words[idx].ori == 0:
                for idy in range(idx, len(self.words)):
                    self.words[idy].toggle()
    
    def __str__(self):
        return "[%s] Words:%s"%(self.__class__.__name__, len(self.words)) 
        
    @staticmethod
    def toWords(stn):
        ''' 開成字段 '''
        if re.search(u"　", stn):
            return stn.split(u"　")
        if re.search(u" ", stn):
            return stn.split(u" ")
      
    @staticmethod
    def skipWindow(seq, max_win=5):
        """
            SKIP Window 算法
        """
        olen = len(seq)
        rterms = []
        for pivot in range(olen):
            left  = (pivot - max_win) if (pivot - max_win) > 0 else 0
            right = (pivot + max_win) if (pivot + max_win) < olen else olen

            for idx in range(left, right):
                if not idx == pivot and (seq[pivot].wanted and seq[idx].wanted):
                    """ 回傳組合，不從順序，從筆劃 """
                    if len(seq[pivot].word[0]) > 0 and len(seq[idx].word[0]) > 0:
                        if seq[pivot].word[0] > seq[idx].word[0]:
                            rterms.append( ( seq[pivot], seq[idx] ) ) 
                        else:
                            rterms.append( ( seq[idx], seq[pivot] ) ) 

        return set(rterms)
 
       

In [7]:
class zOpinion:
    """ 單一評論 """
    def __init__(self, opn):
        self.data = opn
        self.stns = []
        self.word_only = []
        self.proc()
        
    """ 處理評論內容 """
    def proc(self):
        stns = zOpinion.toStn(self.data)
        for stn in stns:
            self.stns.append(zSentence(stn))
            
        self.stat()
        self._doWords4Tag()
        
    """ 評論內的字數統計 """
    def stat(self):
        
        ''' 單一詞統計 '''
        oWords = []
        
        ''' bigram 統計 '''
        oBigrams = []
        oRadiBigrams = []
        for stn in self.stns:
            self.word_only.extend([x.word for x in stn.get_words()])
            oWords.extend([ x.word_best for x in stn.get_wanted()])
#             oBigrams.extend(stn.get_bigrams())
#             oRadiBigrams.extend([ (term2Radi(x), term2Radi(y)) for (x, y) in stn.get_bigrams()])
            
        # 單一詞結果
        self.stat_words = oWords
        self.stat_words_dic = collections.Counter(oWords)
        
        # 雙詞的結果
#         self.stat_bigrams = oBigrams
#         self.stat_bigrams_dic = collections.Counter(oBigrams)
        
        # 雙詞的部首
#         self.stat_radi_bigram = oRadiBigrams
        
    def getOrg(self):
        return self.data['tagged']
        
    """ 取得特定 key 的值 """
    def get(self, key):
        if key in self.getKeys():
            return self.data[key]
        
    def _doWords4Tag(self):
        wrds = []
        for stn in self.stns:
            for word in stn.get_wanted():
                wrds.append(word.get_best()) # here
        self._stn_words = wrds
#         self._stn_radical_pairs = [ term2Radi(x) for x in wrds]

    def getWords4Tag(self):
        return self._stn_words
    
    def getRadicalsPair4Tag(self):
        return self._stn_radical_pairs
    
    
    def __str__(self):
        return "[%s] words:%s"%(self.__class__.__name__, len(self.stat_words_dic.keys()))  

        
    def getPair(self, wanted_terms):
        """ 取得必要的 pair """

#         sorted2Terms = lambda x, y: (x, y) if x > y else (y, x)
#         isInBigram = lambda x,y: sorted2Terms(x,y) if sorted2Terms(x,y) in self.stat_bigrams_dic.keys() else False

        wanted_pairs = [ x for x in self.stat_bigrams_dic.keys() if x[0] in wanted_terms or x[1] in wanted_terms]

        return_pairs = []
        for stn in self.stns:
            stn_terms = list(set([ wrd.word_best for wrd in stn.get_wanted()]))
            for pair in wanted_pairs:
                size = len(set([ x.replace("-", "") for x in stn_terms]).intersection(set(pair)))
                if size >= 1:
                    org_x = "".join([ x for x in stn_terms if x.replace("-", "") == pair[0]])
                    org_y = "".join([ x for x in stn_terms if x.replace("-", "") == pair[1]])
                    return_pairs.append( "%s/%s"%(org_x, org_y) )
            
        return list(set(return_pairs))
    
    @staticmethod
    def toStn(stn):
        ''' 開成字段, [ [word, word], [] ... ] '''
        tagged = u"　 ".join(stn.split("\n"))
        words = zSentence.toWords(tagged)
        stns = []
        stn = []
        for idx in range(len(words)):
            ''' 可能會有空字串 '''
            try:
                (word, tag) = zWord.toElements(words[idx])
                stn.append(words[idx])
                if zTags.isEol(tag):
                    # 如果字串少於1 個字，就跳開
                    if len(stn) >1:
                        stns.append(stn)
                    stn = []
            except: 
                continue
        if len(stn) >1:
            stns.append(stn)

        return stns



# 由此開始

In [8]:
oChi = chilin.chilin()

def cnv2chilin(wrd):
    ''' 使用詞林擴充 '''
    tobj = oChi.getHead(wrd.encode("utf8"), 8)
    if tobj:
        tobj = [ x for x in tobj if (not x[0] in ['A', 'C', 'J', 'K', 'L']) and not x[-1] in ['#']]
        if len(tobj)>0:
#             print wrd, " ".join(tobj)
            grps = [ [ y.decode("utf8") for y in oChi.getWrd(x)] for x in tobj ] 
            return set([item for sublist in grps for item in sublist]) # see http://bit.ly/1FrTjwF
        return set([wrd])
    else:
        return set([wrd])
    
def chilinExt(aSet):
    # 使用辭林擴充
    ext = set()

    if type(aSet) == type(set()):
        for x in aSet:
            flg = False
            if re.search("-$", x):
                flg = True

            cnvted = cnv2chilin(x.replace("-", ""))
            ext.update(["%s"%x if not flg else "%s-"%x for x in cnvted])
    elif type(aSet) == type(u""):
        flg = False
        if re.search("-$", aSet):
            flg = True
        cnvted = cnv2chilin(aSet.replace("-", ""))
        ext.update(["%s"%x if not flg else "%s-"%x for x in cnvted])
        
    return " ".join(ext).replace("-", u"負")

def getChilinTxt(tagged):
    opn = zOpinion(tagged)
#     print len(opn.stns)
    txts = Parallel(n_jobs=-1)(delayed(chilinExt)(set([x.get_best() for x in stn.get_wanted()])) for stn in opn.stns)
#     for stn in opn.stns:
#         print " ".join(stn.stn_org)
# #         print " ".join(["%s, %s\n"%(x.get_best(), " ".join(chilinExt(x.get_best()))) for x in stn.get_wanted()])
#         print chilinExt(set([x.get_best() for x in stn.get_wanted()]))
        
    return " ".join(txts)
# print " ".join(chilinExt([u"錯"]))

In [9]:
oCol = load("./zobj/Taipei_shops_comments.zobj")

In [10]:
print len(oCol)

16908


//1 build dictionary

In [11]:
txts = [ x[1].split(" ") for x in oCol]
dictionary = corpora.Dictionary(txts)
dictionary.filter_tokens(bad_ids=[0])
dictionary.filter_extremes(no_below=5)
dictionary.compactify()
len(dictionary.token2id)

dictionary.save("./zobj/16908_dictionary.mm")

In [12]:
ntusd_hash = load("./zobj/NTUSD_HASH_POS_NEG.zobj")

In [13]:
dictionary = corpora.Dictionary.load("./zobj/16908_dictionary.mm")
print len(dictionary.token2id)

65389


In [15]:
rank_code = lambda x: "P" if int(x)>50 else "N"


print rank_code(oCol[1][2])

N


In [16]:
txts_pos = [ x for x in oCol if rank_code(x[2])=="P"]
txts_neg = [ x for x in oCol if rank_code(x[2])=="N"]


corpus_pos = [ dictionary.doc2bow(x[1].split(" ")) for x in txts_pos]
corpus_neg = [ dictionary.doc2bow(x[1].split(" ")) for x in txts_neg]
print len(corpus_pos), len(corpus_neg)


5420 11488


In [17]:
corpora.MmCorpus.serialize('./zobj/Taipei_shop_corpus_pos.mm', corpus_pos)
corpora.MmCorpus.serialize('./zobj/Taipei_shop_corpus_neg.mm', corpus_neg)

In [18]:
tfidf_pos = models.TfidfModel(corpus_pos)
lsi = models.LsiModel(tfidf_pos[corpus_pos], num_topics=200, id2word=dictionary)

In [19]:
print "\n".join(lsi.print_topics(10))

0.035*"蛋糕" + 0.022*"燔" + 0.022*"麵包" + 0.022*"拉麵" + 0.022*"鬆餅" + 0.021*"巧克力" + 0.021*"炕" + 0.021*"炙" + 0.021*"烤" + 0.021*"烘製"
0.049*"蛋糕" + -0.041*"可觀負" + -0.041*"名不虛傳負" + -0.041*"口碑載道負" + -0.041*"有口皆碑負" + -0.041*"交口稱譽負" + -0.041*"上佳負" + -0.041*"帥負" + -0.041*"名特新優精負" + -0.041*"優負"
0.080*"麻辣" + 0.080*"辣味" + 0.080*"辛" + 0.080*"辣絲絲" + 0.080*"辣乎乎" + 0.080*"辛辣" + -0.076*"蛋糕" + 0.072*"拉麵" + 0.065*"辣" + 0.062*"如狼似虎"
-0.094*"蛋糕" + 0.077*"辣乎乎" + 0.077*"辛" + 0.077*"辣味" + 0.077*"辣絲絲" + 0.077*"麻辣" + 0.076*"辛辣" + 0.060*"鍋" + 0.058*"辣" + 0.052*"拉麵"
-0.046*"拉麵" + -0.043*"臧負" + -0.043*"仁至義盡負" + -0.043*"助人為樂負" + -0.043*"樂善好施負" + -0.043*"善良負" + -0.043*"足以負" + -0.043*"堪負" + -0.043*"得以負" + -0.043*"何嘗不可負"
-0.062*"鬆餅" + 0.054*"拳打腳踢" + 0.054*"毆" + 0.054*"毆鬥" + 0.054*"毆打" + 0.054*"揮拳" + 0.053*"舀" + 0.053*"挹" + 0.052*"交手" + 0.052*"格鬥"
-0.106*"拉麵" + -0.065*"刈" + -0.065*"招攬" + -0.065*"攬客" + -0.065*"招徠" + -0.065*"直拉" + -0.065*"抻" + -0.065*"拉長" + -0.065*"拉縴" + -0.065*"拉杆"
0.062*"刈" + 0.062*"攬客" + 0.062*"招徠" + 0.062*

In [20]:
index = similarities.MatrixSimilarity(lsi[corpus_pos])

In [21]:
mstr = u"不會(D)　辣(VH)　的(DE)　火鍋(Na)"
# tagged = getPOS(mstr.encode("utf8"))
# print tagged
# mstr += " ".join([ x.replace("-", u"負") for x in ntusd_hash["NEG"]])
zOpinion(mstr).getWords4Tag()

[u'\u8fa3-', u'\u706b\u934b-']

In [22]:
proc_txt = [ x.replace("-", u"負") for x in zOpinion(mstr).getWords4Tag()]
print " ".join(proc_txt)
mstr_bow = dictionary.doc2bow(proc_txt)
vec_lsi = lsi[mstr_bow]
sims = index[vec_lsi]

辣負 火鍋負


In [23]:
def clnTxt(mstr):
    return "".join([ x.split("(")[0] for x in mstr.strip().split(u"　") if len( x.split("(") ) >1])


shops_weight = {}
for x in txts_pos[:3]:
    shop_title = x[0]['title']
    if not shops_weight.has_key(shop_title):
        shops_weight[ shop_title ] = { 'lat': x[0]['latitude'],
                                          'lon': x[0]['longitude'],
                                          'adr': x[0]['addr'],
                                          'val': []}
    shops_weight[ shop_title ]['val'].append( x[2])
print shops_weight

{u'27502-\u65b0\u7586\u9905': {'lat': '25.11255726402854', 'adr': u'\u53f0\u5317\u5e02\u58eb\u6797\u5340\u58eb\u6771\u8def266\u5df75\u5f0417\u865f', 'lon': '121.53759574765395', 'val': [u'55', u'55', u'55']}}


In [26]:
import math
 
def distance_on_unit_sphere(lat1, long1, lat2, long2):
    # 以 KM 回傳，原程式碼在 http://goo.gl/JExDdc
    # 回傳是公里
    # Convert latitude and longitude to 
    # spherical coordinates in radians.
    degrees_to_radians = math.pi/180.0
         
    # phi = 90 - latitude
    phi1 = (90.0 - lat1)*degrees_to_radians
    phi2 = (90.0 - lat2)*degrees_to_radians
         
    # theta = longitude
    theta1 = long1*degrees_to_radians
    theta2 = long2*degrees_to_radians
         
    # Compute spherical distance from spherical coordinates.
         
    # For two locations in spherical coordinates 
    # (1, theta, phi) and (1, theta, phi)
    # cosine( arc length ) = 
    #    sin phi sin phi' cos(theta-theta') + cos phi cos phi'
    # distance = rho * arc length
     
    cos = (math.sin(phi1)*math.sin(phi2)*math.cos(theta1 - theta2) + 
           math.cos(phi1)*math.cos(phi2))
    arc = math.acos( cos )
 
    # Remember to multiply arc by the radius of the earth 
    # in your favorite set of units to get length.
    return arc * 6373

def diff_lat_lon(me_loc, target):
    return distance_on_unit_sphere(float(me_loc['lat']), float(me_loc['lon']), 
                                   float(target['lat']), float(target['lon']))

In [27]:
me_loc = { 'lat': 25.041171, "lon":121.565227}
for (x,y) in sorted(enumerate(sims), key=lambda item: -item[1])[:3]:
    target = { 'lat': txts_pos[x][0]['latitude'], 'lon': txts_pos[x][0]['longitude'] }
    
    print diff_lat_lon(me_loc, target), y
    print clnTxt(txts_pos[x][3])

2.64666186361 0.652716
>>看到沒?傳說中的豆腐冰淇淋，真的是超超超超好吃>>滑溜溜的鴨血令人回味無窮，燙在口中暖在心理真的是超美味的啦>>一旁很像飲料的是高湯>>來了來了傳說中的無老鍋，他的麻辣鍋我覺得不辣反而是香麻整體來說真的非常的棒，不過建議四個人去最剛好，因為我跟我朋友兩個人去隨便點一點就吃了２千多塊，貴但是非常值得。		
1.09844636177 0.598129
一家有歷史的老店不能墨守成規,才不會被潮流所淹沒我在有六十幾年歷史的老店~四川吳抄手~,看見年輕人堅守傳統老味道的同時還勇於創新,開發更多新商品,賦予老店新的生命位於台北忠孝東路上的四川吳抄手,第三代老闆是一位年輕漂亮的小姐,年輕的老闆非常用心,以川菜料理手法研發便利美食~川饌之味~讓無法親臨店內品嚐美食的朋友們隨時都能透過宅配的方式吃到吳抄手的美味很榮幸這次能應黑貓探險隊的邀請讓我有機會品嚐６０年老店的美味首先上桌的是鴛鴦鍋,顛覆了我對鴛鴦鍋的印象,一般的鴛鴦鍋不辣的是白湯底,吳抄手的鍋底,因為是用大骨湯加上三十餘種中式香料採純手工傳統方式炒製,左邊是不辣,右邊是麻辣鍋,辣與不辣的鍋底都一樣,差別只在於辣的有放特製辣油吳抄手的麻辣鍋不叫辣,應該叫麻,連不辣的湯底也會有微微的麻,此麻會讓人很容易接受連麻辣鍋也是如此,它是麻喔不是辣,我對麻辣鍋並不是特敢吃,對於吳抄手的麻辣鍋卻很容易接受羊肩肉雪花豬牛小排特製沾醬=黑色的是麻辣醬,黃色的沾醬,它叫麻蛋,是高級麻油加上新鮮蛋黃攪拌而成,只有老饕才知道的喔,沾法是先沾麻辣醬讓肉有鹹味)再沾麻蛋讓肉質更滑嫩)特製滷花生=花生是廚師獨門料理,它是用豬皮下去滷製,會讓花生顯的油油亮亮的從外觀看起來很鹹,實際上並不鹹,還會讓人欲罷不能烏梅汁也是店家的獨門配方,外面的烏梅汁是鹹的不得了,吳抄手的是會讓人回甘,目前還沒有推宅配,只因第二代傳人,堅持要找到合適的瓶子才要推出宅配這些丸類大都是自家工廠生產真的是真材實料,與外面吃的丸類差別很大的,即使久煮也不會影響到口感,表示吳抄手的魚漿是用頂級材料,像花枝腸就真的吃到整塊花枝吳抄手的火鍋,是用炭火烹調,店家說保溫效果較好只是我們習慣用瓦斯爐火以方便調整火侯,但是勒~用炭火無法調整火侯的大小,一時間還真不習慣,老是想用手去關火吃到最後,還能用麵條下去煮麵條也是店家特製的)白麵條的香,裹滿麻辣湯汁

依照店家建立相似度模型後，將特別的字詞與 NTUSD align，最後推出合適的店家。

# 地理位置計算 #

In [ ]:
import urllib
import urllib2
import json


params = {'address': '台北市政府','sensor': 'false'}
url = 'http://maps.googleapis.com/maps/api/geocode/json?' + urllib.urlencode(params)
rawreply = urllib2.urlopen(url).read()
result = json.loads(rawreply)

lat, lng = [(s['formatted_address'],s['geometry']) for s in result['results']][0][1][u'location'].values()
print lat, lng

In [ ]:
distance_on_unit_sphere(23.5852855, 119.61, 23.5852855, 119.6)